In [1]:
import os
import pandas as pd
import sqlite3

from sqlite3 import Error

#### Get current working directory

In [2]:
cwd = os.getcwd()

cwd

'C:\\Users\\randy\\Desktop\\Personal\\github\\demo_sql'

#### Connect to database

In [3]:
def create_connection(db_file):
    
    conn = None
    
    try:
        print(sqlite3.version)
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    
    return conn

In [4]:
conn = create_connection(
    os.path.join(
        cwd,
        'pythonsqlite.db'
    )
)

2.6.0


In [5]:
conn

#### Drop tables if exist from past runs

In [6]:
c = conn.cursor()

for table in ['deposit_marketing', 'month_mapper']:
    c.execute('DROP TABLE IF EXISTS {}'.format(table))
    
c.close()

#### Read local csv into pandas

In [7]:
df = pd.read_csv(
    os.path.join(
        'file://',
        cwd,
        'bank.csv'
    ),
    sep=','
)

#### Analyse schema

In [8]:
df.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

#### Create table for pandas df

In [9]:
columns = []

for col, data_type in df.dtypes.to_dict().items():
    
    col = 'defaulted' if col == 'default' else col
    
    columns.append(
        '{} {}'.format(col, 'integer' if data_type == 'int64' else 'text')
    )

create_tbl_stmt = '{} ({});'.format(
        'CREATE TABLE IF NOT EXISTS deposit_marketing',
        ', '.join(columns)
)

create_tbl_stmt

'CREATE TABLE IF NOT EXISTS deposit_marketing (age integer, job text, marital text, education text, defaulted text, balance integer, housing text, loan text, contact text, day integer, month text, duration integer, campaign integer, pdays integer, previous integer, poutcome text, deposit text);'

In [10]:
def create_table(conn, create_tbl_stmt):
    
    try:
        c = conn.cursor()
        c.execute(create_tbl_stmt)
        c.close()
        
    except Error as e:
        print(e)

In [11]:
create_table(conn, create_tbl_stmt)

#### Check created table

In [12]:
def query_table(conn, query_tbl_stmt):
    
    result = []
    
    try:
        c = conn.cursor()
        c.execute(query_tbl_stmt)
        
        result = pd.DataFrame(
            data=[_ for _ in c],
            columns=[_[0] for _ in c.description]
        )
        
        c.close()
    
    except Error as e:
        print(e)
        
    return result

In [13]:
query_table(conn, 'select * from sqlite_master')

,type,name,tbl_name,rootpage,sql
0,table,deposit_marketing,deposit_marketing,2,"CREATE TABLE deposit_marketing (age integer, j..."


In [14]:
query_table(conn, 'select * from deposit_marketing limit 5')

,age,job,marital,education,defaulted,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit


#### Insert records

In [15]:
records = df.to_records(index=False).tolist()

c = conn.cursor()
c.executemany(
    'INSERT INTO deposit_marketing VALUES ({})'.format(','.join(['?' for _ in range(0, len(df.columns))])),
    records
)
c.close()
conn.commit()

#### Query inserted records

In [16]:
query_table(conn, 'select * from deposit_marketing limit 5')

,age,job,marital,education,defaulted,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


#### Distinct value in poutcome

In [17]:
query_table(conn, 'select distinct poutcome from deposit_marketing')

,poutcome
0,unknown
1,other
2,failure
3,success


#### Standardising value of poutcome

In [18]:
c = conn.cursor()
c.execute(
    '''
        update deposit_marketing
        set
            poutcome == 'unknown'
        where
            poutcome == 'other'
    '''
)
c.close()
conn.commit()

In [19]:
query_table(conn, 'select distinct poutcome from deposit_marketing')

,poutcome
0,unknown
1,failure
2,success


#### Balances summary by education and marital

In [20]:
query_table(
    conn,
    '''
    select
        education,
        marital,
        count(*) as tot_population,
        sum(balance) as tot_balances,
        avg(balance) as avg_balances,
        min(balance) as min_balances,
        max(balance) as max_balances
    from deposit_marketing
    group by
        education,
        marital
    order by education asc, marital desc
    '''
)

,education,marital,tot_population,tot_balances,avg_balances,min_balances,max_balances
0,primary,single,197,271800,1379.695431,-887,26965
1,primary,married,1099,1723931,1568.636033,-1489,66653
2,primary,divorced,204,288816,1415.764706,-779,37127
3,secondary,single,1704,1942320,1139.859155,-1139,56831
4,secondary,married,3120,4500996,1442.626923,-1965,81204
5,secondary,divorced,652,656210,1006.457055,-934,12039
6,tertiary,single,1460,2637784,1806.701370,-880,36252
7,tertiary,married,1843,3433041,1862.746066,-6847,51439
8,tertiary,divorced,386,738586,1913.435233,-2282,52587
9,unknown,single,157,274720,1749.808917,-461,45248


#### Top 3 balance by marital

In [21]:
query_table(
    conn,
    '''
    select * from (
        select
            marital,
            job,
            age,
            balance,
            dense_rank() over(
                partition by marital
                order by balance desc
            ) as r1
        from deposit_marketing
    )
    where r1 < 4
    order by marital desc
    '''
)

,marital,job,age,balance,r1
0,single,admin.,43,56831,1
1,single,technician,39,45248,2
2,single,management,29,36252,3
3,married,retired,84,81204,1
4,married,retired,84,81204,1
5,married,blue-collar,52,66653,2
6,married,entrepreneur,56,51439,3
7,divorced,self-employed,61,52587,1
8,divorced,self-employed,61,52587,1
9,divorced,retired,75,37127,2


#### Create another table

In [22]:
create_table(
    conn,
    '''
    CREATE TABLE IF NOT EXISTS month_mapper (
        month text primary key,
        month_num integer
    )
    '''
)

In [23]:
query_table(conn, 'select * from sqlite_master')

,type,name,tbl_name,rootpage,sql
0,table,deposit_marketing,deposit_marketing,2,"CREATE TABLE deposit_marketing (age integer, j..."
1,table,month_mapper,month_mapper,7,CREATE TABLE month_mapper (\n month tex...
2,index,sqlite_autoindex_month_mapper_1,month_mapper,8,None


In [24]:
records = [
    ('jan', 1),
    ('feb', 2),
    ('mar', 3),
    ('apr', 4),
    ('may', 5),
    ('jun', 6),
    ('jul', 7),
    ('aug', 8),
    ('sep', 9),
    ('oct', 10),
    ('nov', 11),
    ('dec', 12)
]

c = conn.cursor()
c.executemany(
    'INSERT INTO month_mapper VALUES (?,?)',
    records
)
c.close()
conn.commit()

In [25]:
query_table(conn, 'select * from month_mapper')

,month,month_num
0,jan,1
1,feb,2
2,mar,3
3,apr,4
4,may,5
5,jun,6
6,jul,7
7,aug,8
8,sep,9
9,oct,10


#### Join dataframes

In [26]:
query_table(
    conn,
    '''
    select
        dm.*,
        mm.*
    from deposit_marketing dm
    inner join month_mapper mm
    on
        dm.month = mm.month
    limit 5
    '''
)

,age,job,marital,education,defaulted,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,month,month_num
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes,may,5
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes,may,5
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes,may,5
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes,may,5
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes,may,5


#### Close connection

In [27]:
conn.close()